In [83]:
from statsmodels.tsa.stattools import adfuller

data = [5,6,7,8,10,11,12,14,15,17]
adfuller(data)

(1.5894388284780538,
 0.9978255428896379,
 2,
 7,
 {'1%': -4.9386902332361515,
  '5%': -3.477582857142857,
  '10%': -2.8438679591836733},
 7.68503185107909)

In [84]:
import pandas as pd
data = pd.DataFrame({'vect':[5,6,7,8,10,11,12,14,15,17]})
data['vect_s1'] = data['vect'].shift(1)
data['vect_s2'] = data['vect'].shift(2)
data['shift_1'] = data['vect']-data['vect_s2']
data['shift_1_shift'] = data['shift_1'].shift(-1)

In [85]:
data[['shift_1_shift', 'shift_1']][2:-1]

,shift_1_shift,shift_1
2,2.0,2.0
3,3.0,2.0
4,3.0,3.0
5,2.0,3.0
6,3.0,2.0
7,3.0,3.0
8,3.0,3.0


In [86]:
import numpy as np
from sklearn.linear_model import LinearRegression
x = np.array(data['shift_1_shift'][2:-1]).reshape(-1,1)
y = np.array(data['shift_1'][2:-1])
reg = LinearRegression().fit(x, y)
print(reg.score(x, y))
print(reg.coef_)
print(reg.intercept_)

0.008333333333333193
[0.1]
2.3000000000000003


In [182]:
pd.options.mode.chained_assignment = None

In [251]:
data_holt = pd.DataFrame({'data': [100, 105, 108, 107, 110]}, columns = ['data', 'l_step', 'b_step'])
data_holt['b_step'].loc[0] = 5
data_holt['l_step'].loc[0] = 100
alfa = 0.5
beta = 0.4


In [252]:
def HoltPredict():
    for i in range(len(data_holt)):
        if i > 0: 
            data_holt['l_step'].loc[i] = alfa * data_holt['data'].loc[i] + (1 - alfa) * (data_holt['l_step'].loc[i-1] + data_holt['b_step'].loc[i-1])
            data_holt['b_step'].loc[i] = beta * (data_holt['l_step'].loc[i]-data_holt['l_step'].loc[i-1]) + (1 - beta) * data_holt['b_step'].loc[i-1]
    pass
HoltPredict()
data_holt

,data,l_step,b_step
0,100,100,5
1,105,105.0,5.0
2,108,109.0,4.6
3,107,110.3,3.28
4,110,111.79,2.564


In [253]:
import numpy as np
from sklearn.linear_model import LinearRegression
x = np.array(data_holt[['l_step', 'b_step']])
y = np.array(data_holt['data'])
reg = LinearRegression().fit(x, y)
print(reg.score(x, y))
print(reg.coef_)
print(reg.intercept_)

0.9568611629903695
[0.90078248 0.64996624]
6.76232219804767


In [254]:
alfa = 0.90078248
beta = 6.76232219804767
HoltPredict()
data_holt

,data,l_step,b_step
0,100,100,5
1,105,105.0,5.0
2,108,108.198435,-7.182763
3,107,106.40625,29.270059
4,110,112.54754,-127.134132


In [274]:
data_hv = pd.DataFrame({'data': [100, 120, 110, 130, 105, 125, 115, 135]}, columns = ['data', 'l_step', 'b_step', 's_step'])
data_hv['b_step'].loc[0] = 5
data_hv['l_step'].loc[0] = 100
for i in range(4):
    data_hv['s_step'].loc[i] = 1
alfa = 0.5
beta = 0.4
gamma = 0.6

In [275]:
def HoltVinPredict():
    for i in range(len(data_hv)):
        if (i > 0) and (i < 4): 
            data_hv['l_step'].loc[i] = alfa * data_hv['data'].loc[i] / data_hv['s_step'].loc[i] + (1 - alfa) * (data_hv['l_step'].loc[i-1] + data_hv['b_step'].loc[i-1])
            data_hv['b_step'].loc[i] = beta * (data_hv['l_step'].loc[i] - data_hv['l_step'].loc[i-1]) + (1 - beta) * data_hv['b_step'].loc[i-1]
        if i > 3: 
            data_hv['l_step'].loc[i] = alfa * data_hv['data'].loc[i] / data_hv['s_step'].loc[i-4] + (1 - alfa) * (data_hv['l_step'].loc[i-1] + data_hv['b_step'].loc[i-1])
            data_hv['b_step'].loc[i] = beta * (data_hv['l_step'].loc[i] - data_hv['l_step'].loc[i-1]) + (1 - beta) * data_hv['b_step'].loc[i-1]    
            data_hv['s_step'].loc[i] = gamma * data_hv['data'].loc[i] / (data_hv['l_step'].loc[i] + data_hv['b_step'].loc[i]) + (1 - gamma) * data_hv['s_step'].loc[i-4]
    pass
HoltVinPredict()
data_hv

,data,l_step,b_step,s_step
0,100,100,5,1
1,120,112.5,8.0,1
2,110,115.25,5.9,1
3,130,125.575,7.67,1
4,105,119.1225,2.021,0.920044
5,125,123.07175,2.7923,0.995881
6,115,120.432025,0.61949,0.970005
7,135,128.025757,3.409187,1.016274


In [270]:
import numpy as np
from sklearn.linear_model import LinearRegression
x = np.array(data_hv[['l_step', 'b_step']])
y = np.array(data_hv['data'])
reg = LinearRegression().fit(x, y)
print(reg.score(x, y))
print(reg.coef_)
print(reg.intercept_)

0.3333342897571564
[ 2.16102902e-20 -3.33287733e-17]
114.9878345975556


In [276]:
alfa = 2.16102902e-20
beta = 114.9878345975556
HoltVinPredict()
data_hv

,data,l_step,b_step,s_step
0,100,100,5,1
1,120,105.0,5.0,1
2,110,110.0,5.0,1
3,130,115.0,5.0,1
4,105,120.0,5.0,0.904
5,125,125.0,5.0,0.976923
6,115,130.0,5.0,0.911111
7,135,135.0,5.0,0.978571
